In [ ]:
import tensorflow as tf
import pandas as pd
import numpy as np

In [ ]:
with open("/content/drive/MyDrive/Sherlock Holmes.txt", "r") as file:
    text_data = file.read()

In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense,Dropout,Bidirectional
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical


In [ ]:
# Preprocess the text
tokenizer = Tokenizer()
tokenizer.fit_on_texts([text_data])
total_words = len(tokenizer.word_index) + 1


In [ ]:
total_words

In [ ]:
# Create input sequences and targets
input_sequences = []
for line in text_data.split('\n'):
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)

In [ ]:
# Pad sequences to have equal length
max_length = max([len(seq) for seq in input_sequences])
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_length, padding='pre'))

In [ ]:
# Split input sequences into input and target
X, y = input_sequences[:, :-1], input_sequences[:, -1]
y = to_categorical(y, num_classes=total_words)

In [ ]:
X.shape,y.shape

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Embedding, LSTM, Dense, Bidirectional, Dropout

# Define the optimizer with a specific learning rate
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)

# Define the model architecture
model = tf.keras.Sequential([
    # Embedding layer to convert words to dense vectors
    Embedding(input_dim=total_words, output_dim=300, input_length=max_length-1),

    # Bidirectional LSTM layer to capture bidirectional context
    Bidirectional(LSTM(units=256, return_sequences=True)),

    # Dropout layer to prevent overfitting
    Dropout(0.5),

    # Bidirectional LSTM layer for deeper context understanding
    Bidirectional(LSTM(units=128)),

    # Dropout layer for regularization
    Dropout(0.5),

    # Dense layer with softmax activation for prediction
    Dense(total_words, activation='softmax')
])

# Compile the model with the specified optimizer
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])



In [ ]:
model.summary()

In [ ]:
model.fit(X,y,epochs = 50)

In [ ]:
model.save('nxt_word.h5')


In [ ]:
text = 'It '
for a in range(30):
  tok_text = tokenizer.texts_to_sequences([text])[0]
  pad_text = pad_sequences([tok_text],17,padding = 'pre')
  pred = np.argmax(model.predict(pad_text))
  for k,v in tokenizer.word_index.items():
    if v == pred:
      print(f'Next word after "{text}" is: ',k)
      text = text + " " + k